In [32]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from preprocessing import load_and_preprocess_data

from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
padded_docs, artists_onehot_encoded, vocab_size, max_length = load_and_preprocess_data("/content/drive/MyDrive/UU/Pattern recognition/spotify_millsongdata.csv")

In [6]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, artists_onehot_encoded, test_size=0.2, random_state=42)

In [44]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [45]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_length, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=max_length, output_dim=embed_dim)

    def call(self, x):
        max_length = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_length, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [49]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(artists_onehot_encoded.shape[1], activation="softmax")(x)

transformer = keras.Model(inputs=inputs, outputs=outputs)

In [50]:
transformer.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 950)]             0         
                                                                 
 token_and_position_embeddin  (None, 950, 32)          978816    
 g_5 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block_2 (Transf  (None, 950, 32)          10656     
 ormerBlock)                                                     
                                                                 
 global_average_pooling1d_2   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_10 (Dropout)        (None, 32)                0   

In [51]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5,
                                                    restore_best_weights=True,
                                                    verbose=1)]

history = transformer.fit(
    X_train,
    y_train,
    validation_split=0.1,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
    use_multiprocessing=True
)

Epoch 1/200


InvalidArgumentError: ignored